LSE Data Science Institute | DS105A (2023/24) | Week 10

# 🗓️ Week 10: Databases + Data reshaping + Basics of Text Mining

Theme: Cleaning and reshaping data

**LAST UPDATED:** 29 November 2023

**AUTHOR:** Dr [Jon Cardoso-Silva](https://jonjoncardoso.github.io)

-----


⚙️ **Setup**

In [1]:
import re
# My custom Python package
import discordia

import pandas as pd
import tqdm.notebook as tqdm

from datetime import datetime
from selenium import webdriver
from sqlalchemy import create_engine

from discordia.webscraping.twfy import build_url, scrape_debate_sections, get_speeches

The percent sign is a feature of jupyter, not python itself. It allows us to run **magic commands**.

In [2]:
%load_ext sql
%config SqlMagic.autocommit=False # for engines that do not support autommit

# 1. Collect Data

Example of calling a simple function from my package:

In [3]:
date_obj = datetime(2023, 11, 15)

build_url(date_obj)

'https://www.theyworkforyou.com/debates/?d=2023-11-15'

**Start Selenium:**

In [4]:
driver = webdriver.Firefox()

curr_url = build_url(date_obj)
driver.get(curr_url)

## 1.1 Collect debates


Collecting data from the web using Selenium:

In [5]:
df_debates = scrape_debate_sections(driver, curr_url)
df_debates

,debate_id,debate_excerpt,url,title,section,section_excerpt
0,2023-11-15b.629.0,I can now announce the arrangements for the el...,https://www.theyworkforyou.com/debates/?id=202...,Speaker’s Statement,None,None
1,2023-11-15b.629.5,What steps she is taking with Cabinet colleagu...,https://www.theyworkforyou.com/debates/?id=202...,Online Fraud,"Science, Innovation and Technology",The Secretary of State was asked—
2,2023-11-15b.630.3,What recent discussions she has had with (a) O...,https://www.theyworkforyou.com/debates/?id=202...,Telecoms Network Replacement,"Science, Innovation and Technology",The Secretary of State was asked—
3,2023-11-15b.631.1,What steps her Department is taking to tackle ...,https://www.theyworkforyou.com/debates/?id=202...,AI-generated Content: Social Media,"Science, Innovation and Technology",The Secretary of State was asked—
4,2023-11-15b.632.5,What steps the Government are taking to regula...,https://www.theyworkforyou.com/debates/?id=202...,AI Regulation,"Science, Innovation and Technology",The Secretary of State was asked—
5,2023-11-15b.633.6,What steps her Department is taking to improve...,https://www.theyworkforyou.com/debates/?id=202...,Rural Connectivity,"Science, Innovation and Technology",The Secretary of State was asked—
6,2023-11-15b.634.4,What steps she is taking with Cabinet colleagu...,https://www.theyworkforyou.com/debates/?id=202...,Net Zero Technologies: University Research,"Science, Innovation and Technology",The Secretary of State was asked—
7,2023-11-15b.635.5,If she will make a statement on her department...,https://www.theyworkforyou.com/debates/?id=202...,Topical Questions,"Science, Innovation and Technology",The Secretary of State was asked—
8,2023-11-15b.638.4,If he will list his official engagements for W...,https://www.theyworkforyou.com/debates/?id=202...,Engagements,Prime Minister,The Prime Minister was asked—
9,2023-11-15b.642.0,What assessment he has made of recent trends i...,https://www.theyworkforyou.com/debates/?id=202...,West Midlands: Economic Growth,Prime Minister,The Prime Minister was asked—


I won't need Selenium anymore (listen to the lecture to understand why), so I will just close it.


In [7]:
# Thanks, Selenium. Your time is done.
driver.quit()

KeyboardInterrupt: 

## 1.2 Collect speeches

In [8]:
df_speeches = pd.concat([get_speeches(debate_url) for debate_url in df_debates['url']])

In [9]:
df_speeches.head()

,debate_id,speech_id,speaker_id,speaker_position,speech_html,speech_raw_text
0,2023-11-15b.629.0,g629.1,10295,"Speaker of the House of Commons, Chair, Speake...","<p pid=""b629.1/1"">\n I can now announce the ar...",I can now announce the arrangements for the el...
0,2023-11-15b.629.5,g629.6,10580,"Conservative, New Forest West","<p pid=""b629.6/1"" qnum=""900097"">\n What steps ...",What steps she is taking with Cabinet colleagu...
1,2023-11-15b.629.5,g629.7,25847,Parliamentary Under Secretary of State (Depart...,"<p pid=""b629.7/1"">\n Tackling fraud is a prior...",Tackling fraud is a priority for this Governme...
2,2023-11-15b.629.5,g629.8,10580,"Conservative, New Forest West","<p pid=""b629.8/1"">\n What will companies actua...",What will companies actually have to do under ...
3,2023-11-15b.629.5,g629.9,25847,Parliamentary Under Secretary of State (Depart...,"<p pid=""b629.9/1"">\n All companies in scope of...",All companies in scope of the Act will need to...


# 2. Automated data collection

(For explanation, watch the lecture)

In [10]:
start_date = datetime(2023, 11, 1)
end_date = datetime(2023, 11, 29)

all_urls = [build_url(date_obj) for date_obj in pd.date_range(start_date, end_date)]
all_urls

['https://www.theyworkforyou.com/debates/?d=2023-11-01',
 'https://www.theyworkforyou.com/debates/?d=2023-11-02',
 'https://www.theyworkforyou.com/debates/?d=2023-11-03',
 'https://www.theyworkforyou.com/debates/?d=2023-11-04',
 'https://www.theyworkforyou.com/debates/?d=2023-11-05',
 'https://www.theyworkforyou.com/debates/?d=2023-11-06',
 'https://www.theyworkforyou.com/debates/?d=2023-11-07',
 'https://www.theyworkforyou.com/debates/?d=2023-11-08',
 'https://www.theyworkforyou.com/debates/?d=2023-11-09',
 'https://www.theyworkforyou.com/debates/?d=2023-11-10',
 'https://www.theyworkforyou.com/debates/?d=2023-11-11',
 'https://www.theyworkforyou.com/debates/?d=2023-11-12',
 'https://www.theyworkforyou.com/debates/?d=2023-11-13',
 'https://www.theyworkforyou.com/debates/?d=2023-11-14',
 'https://www.theyworkforyou.com/debates/?d=2023-11-15',
 'https://www.theyworkforyou.com/debates/?d=2023-11-16',
 'https://www.theyworkforyou.com/debates/?d=2023-11-17',
 'https://www.theyworkforyou.co

In [11]:
driver = webdriver.Firefox()

df_debates = pd.concat([scrape_debate_sections(driver, url) for url in tqdm.tqdm(all_urls)])

driver.quit()

  0%|          | 0/29 [00:00<?, ?it/s]

In [27]:
df_debates.shape

(322, 6)

In [29]:
df_debates.tail(5)

,debate_id,debate_excerpt,url,title,section,section_excerpt
14,2023-11-29a.1019.1,"Motion made, and Question put forthwith ( Stan...",https://www.theyworkforyou.com/debates/?id=202...,Delegated Legislation,None,None
15,2023-11-29a.1020.2,"Ordered, That Private Members’ Bills shall hav...",https://www.theyworkforyou.com/debates/?id=202...,Business of the House (Private Members’ Bills),None,None
16,2023-11-29a.1020.4,6.17 pm,https://www.theyworkforyou.com/debates/?id=202...,Petition - Ceasefire in Palestine,None,None
17,2023-11-29a.1021.0,"Motion made, and Question proposed, That this ...",https://www.theyworkforyou.com/debates/?id=202...,Refugee Family Reunion Routes: Sudan,None,None
18,2023-11-29a.1033.0,None,https://www.theyworkforyou.com/debates/?id=202...,Deferred Division,None,None


**Get speeches of all debates**

In [15]:
df_speeches = pd.concat([get_speeches(debate_url) for debate_url in tqdm.tqdm(df_debates['url'])])

  0%|          | 0/322 [00:00<?, ?it/s]

In [30]:
df_speeches.shape

(4309, 6)

# 3. Databases

(For explanation, watch the lecture)

In [17]:
# A little trick to find out how big to set my VARCHAR columns in the database.
df_speeches.apply(lambda x: max([len(xx) for xx in x]), axis=0)

debate_id              18
speech_id               7
speaker_id              5
speaker_position      531
speech_html         53137
speech_raw_text     42564
dtype: int64

In [18]:
# Get a template schema to start with.
print(pd.io.sql.get_schema(df_speeches, 'speeches'))

CREATE TABLE "speeches" (
"debate_id" TEXT,
  "speech_id" TEXT,
  "speaker_id" TEXT,
  "speaker_position" TEXT,
  "speech_html" TEXT,
  "speech_raw_text" TEXT
)


## 3.1 Create database and tables

Create a database from within Python:

In [19]:
# Create a database engine using SQLAlchemy
engine = create_engine('sqlite:///../data/discordia.db', echo=False)

# Why? Read: https://stackoverflow.com/a/71685414/843365
with engine.connect() as conn:
    pass

In [20]:
%sql sqlite:///../data/discordia.db --alias discordia

Connecting to 'discordia'

**Create debates table with controlled data types**

TABLE `debates`

In [21]:
%%sql discordia

CREATE TABLE debates (
    "debate_id" VARCHAR(20),
    "debate_excerpt" TEXT,
    "url" VARCHAR(100),
    "title" VARCHAR(100),
    "section" VARCHAR(100),
    "section_excerpt" TEXT
)


RuntimeError: (sqlite3.OperationalError) table debates already exists
[SQL: CREATE TABLE debates (
    "debate_id" VARCHAR(20),
    "debate_excerpt" TEXT,
    "url" VARCHAR(100),
    "title" VARCHAR(100),
    "section" VARCHAR(100),
    "section_excerpt" TEXT
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


TABLE `speeches`

In [22]:
%%sql discordia

CREATE TABLE "speeches" (
    "debate_id" VARCHAR(20),
    "speech_id" VARCHAR(10),
    "speaker_id" CHAR(5),
    "speaker_position" TEXT,
    "speech_html" TEXT,
    "speech_raw_text" TEXT
)

RuntimeError: (sqlite3.OperationalError) table "speeches" already exists
[SQL: CREATE TABLE "speeches" (
    "debate_id" VARCHAR(20),
    "speech_id" VARCHAR(10),
    "speaker_id" CHAR(5),
    "speaker_position" TEXT,
    "speech_html" TEXT,
    "speech_raw_text" TEXT
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


**Move data to database**

In [23]:
df_debates.to_sql('debates', engine, if_exists='append', index=False)
df_speeches.to_sql('speeches', engine, if_exists='append', index=False)

4309

## 3.2 Read from SQL

In [37]:
print(df_speeches.columns)
print(df_debates.columns)

Index(['debate_id', 'speech_id', 'speaker_id', 'speaker_position',
       'speech_html', 'speech_raw_text'],
      dtype='object')
Index(['debate_id', 'debate_excerpt', 'url', 'title', 'section',
       'section_excerpt'],
      dtype='object')


In [40]:
pd.merge(left=df_speeches, right=df_debates,
         left_on= ["debate_id"], right_on=["debate_id"]
         )

,debate_id,speech_id,speaker_id,speaker_position,speech_html,speech_raw_text,debate_excerpt,url,title,section,section_excerpt
0,2023-11-07d.4.0,g4.1,10295,"Speaker of the House of Commons, Chair, Speake...","<p pid=""d4.1/1"">\n I have to acquaint the Hous...",I have to acquaint the House that this House h...,I have to acquaint the House that this House h...,https://www.theyworkforyou.com/debates/?id=202...,King’s Speech,None,None
1,2023-11-07d.7.0,g7.2,10295,"Speaker of the House of Commons, Chair, Speake...","<p pid=""d7.2/1"">\n Before I call the mover and...","Before I call the mover and seconder, I want t...",None,https://www.theyworkforyou.com/debates/?id=202...,Debate on the Address,None,None
2,2023-11-07d.7.0,g7.3,11804,"Chair, Environment, Food and Rural Affairs Com...","<p pid=""d7.3/1"">\n I beg to move,\n</p>\n<p cl...","I beg to move,\n\nThat an humble Address be pr...",None,https://www.theyworkforyou.com/debates/?id=202...,Debate on the Address,None,None
3,2023-11-07d.7.0,g10.0,25838,"Conservative, Stroud","<p pid=""d10.0/1"">\n It is an honour to second ...",It is an honour to second the Loyal Address an...,None,https://www.theyworkforyou.com/debates/?id=202...,Debate on the Address,None,None
4,2023-11-07d.7.0,g13.0,10295,"Speaker of the House of Commons, Chair, Speake...","<p pid=""d13.0/1"">\n I call the\n <a class=""glo...",I call the Leader of the Opposition.,None,https://www.theyworkforyou.com/debates/?id=202...,Debate on the Address,None,None
...,...,...,...,...,...,...,...,...,...,...,...
4304,2023-11-29a.1021.0,g1029.1,25227,The Minister for Immigration,"<p pid=""a1029.1/1"">\n In most cases, unfortuna...","In most cases, unfortunately, individuals will...","Motion made, and Question proposed, That this ...",https://www.theyworkforyou.com/debates/?id=202...,Refugee Family Reunion Routes: Sudan,None,None
4305,2023-11-29a.1021.0,g1029.2,25301,"Scottish National Party, Cumbernauld, Kilsyth ...","<p pid=""a1029.2/1"">\n rose—\n</p>\n",rose—,"Motion made, and Question proposed, That this ...",https://www.theyworkforyou.com/debates/?id=202...,Refugee Family Reunion Routes: Sudan,None,None
4306,2023-11-29a.1021.0,g1029.3,25227,The Minister for Immigration,"<p pid=""a1029.3/1"">\n Before I\n <a class=""glo...","Before I give way to the hon. Gentleman, I wil...","Motion made, and Question proposed, That this ...",https://www.theyworkforyou.com/debates/?id=202...,Refugee Family Reunion Routes: Sudan,None,None
4307,2023-11-29a.1021.0,g1030.0,25301,"Scottish National Party, Cumbernauld, Kilsyth ...","<p pid=""a1030.0/1"">\n I am grateful that the\n...",I am grateful that the Minister is open to hav...,"Motion made, and Question proposed, That this ...",https://www.theyworkforyou.com/debates/?id=202...,Refugee Family Reunion Routes: Sudan,None,None


In [44]:
query = """
SELECT
    debates.debate_id,
    speeches.speech_id,
    speeches.speaker_id,
    debates.title
FROM
    speeches
LEFT JOIN
    debates    
"""

pd.read_sql(query, engine)

,debate_id,speech_id,speaker_id,title
0,2023-11-07d.1.0,g4.1,10295,Message to Attend His Majesty
1,2023-11-07d.3.0,g4.1,10295,Outlawries Bill
2,2023-11-07d.4.0,g4.1,10295,King’s Speech
3,2023-11-07d.7.0,g4.1,10295,Debate on the Address
4,2023-11-07d.105.0,g4.1,10295,Dangerous Driving: Unduly Lenient Sentence Scheme
...,...,...,...,...
5549987,2023-11-29a.1019.1,g1030.1,25227,Delegated Legislation
5549988,2023-11-29a.1020.2,g1030.1,25227,Business of the House (Private Members’ Bills)
5549989,2023-11-29a.1020.4,g1030.1,25227,Petition - Ceasefire in Palestine
5549990,2023-11-29a.1021.0,g1030.1,25227,Refugee Family Reunion Routes: Sudan


In [35]:
query = """
SELECT
    debate_id,
    COUNT(*) as num_speeches
FROM
    speeches
GROUP BY
    speeches.debate_id
"""

pd.read_sql(query, engine)

,debate_id,num_speeches
0,2023-11-07d.105.0,12
1,2023-11-07d.4.0,2
2,2023-11-07d.7.0,186
3,2023-11-08d.113.0,2
4,2023-11-08d.114.0,96
...,...,...
215,2023-11-29a.823.8,16
216,2023-11-29a.824.9,138
217,2023-11-29a.837.0,86
218,2023-11-29a.846.0,24


## 3.3 pandas vs SQL

Question: How many utterances were said by each speaker?

In [33]:
(
    df_speeches.groupby('speaker_id')
               .apply(lambda x: pd.Series({'num_utterances': len(x)}))
               .sort_values('num_utterances', ascending=False)
               .reset_index()
               .head(10)
)

,speaker_id,num_utterances
0,11115,233
1,10295,188
2,25376,100
3,24938,97
4,25428,96
5,11859,85
6,10648,63
7,13864,56
8,25227,53
9,25806,50


In [26]:
%%sql discordia

-- How many utterances were said by each speaker?
SELECT 
    speaker_id, 
    COUNT(*) AS num_utterances 
FROM 
    speeches 
GROUP BY speaker_id
ORDER BY
    num_utterances DESC
LIMIT 10

speaker_id,num_utterances
11115,466
10295,376
25376,200
24938,194
25428,192
11859,170
10648,126
13864,112
25227,106
25806,100


In [45]:
query = """
SELECT 
    speaker_id, 
    COUNT(*) AS num_utterances 
FROM 
    speeches 
GROUP BY speaker_id
ORDER BY
    num_utterances DESC
LIMIT 10
"""

pd.read_sql(query, engine)

,speaker_id,num_utterances
0,11115,466
1,10295,376
2,25376,200
3,24938,194
4,25428,192
5,11859,170
6,10648,126
7,13864,112
8,25227,106
9,25806,100
